In [6]:
%load_ext autoreload
%autoreload 2

import os
from glob import glob 
import cv2
import matplotlib.pyplot as plt 
import numpy as np
import PIL

from BBS_pp_utils import json_to_arr
import BBS_pp_utils as bbpp
import data_preprocessing as dp
from data_utils import Loader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[('frame', '<i8'), ('xl_hand', '<f8'), ('yl_hand', '<f8'), ('xl_elbow', '<f8'), ('yl_elbow', '<f8'), ('xl_shoulder', '<f8'), ('yl_shoulder', '<f8'), ('xr_shoulder', '<f8'), ('yr_shoulder', '<f8'), ('xr_elbow', '<f8'), ('yr_elbow', '<f8'), ('xr_hand', '<f8'), ('yr_hand', '<f8'), ('xhead', '<f8'), ('yhead', '<f8'), ('xneck', '<f8'), ('yneck', '<f8'), ('xpelvis', '<f8'), ('ypelvis', '<f8'), ('xl_foot', '<f8'), ('yl_foot', '<f8'), ('xl_knee', '<f8'), ('yl_knee', '<f8'), ('xl_hip', '<f8'), ('yl_hip', '<f8'), ('xr_hip', '<f8'), ('yr_hip', '<f8'), ('xr_knee', '<f8'), ('yr_knee', '<f8'), ('xr_foot', '<f8'), ('yr_foot', '<f8')]

In [12]:
def sk_viewer(json_to_arr_list, jpg, name=None, class_name=None, idx=0, save=1):
    '''
    skeleton viewer function 
    parameters
    ----------
    json_to_arr_list : json_to_arr_list function result
    jpg_list : Full frame list
    idx : Frame index that you want to see
    image save = 1 / not image save = 0
    '''
    left_arms = ['l_shoulder', 'l_elbow', 'pelvis']
    right_arms = ['l_hand', 'r_shoulder',  'r_elbow', 'pelvis']
    body = ['l_hand','l_shoulder', 'r_shoulder', 'r_hip', 'l_hip', 'l_shoulder']
    leg = ['r_foot', 'r_knee', 'r_hip', 'l_hip', 'l_knee', 'l_foot']
    ii = [left_arms, right_arms, body, leg]

    fig, ax = plt.subplots(figsize=(16,9))
    im = plt.imread(jpg)
    
    # 스켈레톤과 RGB가 이미지 영역도 다르고 해상도도 달라서 맞춰줌 
    xsk = ysk = 512
    scale = 1280/512
        
    im = cv2.resize(im, dsize=(0, 0), fx=1, fy=1, )
    ax.imshow(im, zorder=1)
    for color_idx, i in enumerate(json_to_arr_list):
        if color_idx == 0: 
            color = 'tab:blue'
        elif color_idx == 1:
            color = 'tab:orange'
        else:
            color = 'tab:green'
            
        for j in ii:
            ax.plot([(i['x'+sa][idx] - 0.5*xsk)*scale + 640 for sa in j if i['x'+sa][idx] !=0],
                    [(i['y'+sa][idx] - 0.5*ysk)*scale + 420 for sa in j if i['x'+sa][idx] !=0], lw=5, color=color)
    if save == 1:
        os.makedirs(f'/home/eckim/workspace/fhe_etri/Data/BBS/Sample_BBS/BBS/e_image_plot/{class_name}', exist_ok=True)
        plt.tight_layout()
        plt.savefig(f'/home/eckim/workspace/fhe_etri/Data/BBS/Sample_BBS/BBS/e_image_plot/{class_name}/{name}_{idx}.jpg')
    plt.close()
    #plt.show()

In [8]:
dtypes = bbpp.get_dtypes(ignored_joints=[])

ll = Loader(dtypes=dtypes)

#ll.skel_fn("a", 29, 1, 1)
#fnl = np.array(ll.get_jpg_list())
#sk = ll.skel_fn("a", 29, 4, 4)

In [9]:
root = '/home/eckim/workspace/fhe_etri/Data/BBS/Sample_BBS/BBS'
id_list = glob(root+'/frame_1_idx_e/*/*/*/*.jpg')
id_list.sort()
print(len(id_list))
print(id_list[1])

5863
/home/eckim/workspace/fhe_etri/Data/BBS/Sample_BBS/BBS/frame_1_idx_e/1/0/021/0001.jpg


In [11]:
npy_file = glob(root+'/npy_e/*/*/*/*0.npy')
npy_file.sort()
print(len(npy_file))
print(npy_file[1])

11677
/home/eckim/workspace/fhe_etri/Data/BBS/Sample_BBS/BBS/npy_e/1/0/020/e_020_1_0_0.npy


In [ ]:
for i in id_list:
    _, _, class_name, score, ID, _ = i.split(root)[1].split('/')
    #print(class_name, score, ID)
    arrs = []
    for j in range(len(glob(root+f'/npy_e/{class_name}/{score}/{ID}/e*.npy'))):
        arr = np.load(root+f'/npy_e/{class_name}/{score}/{ID}/e_{ID}_{class_name}_{score}_{j}.npy')
        arrs.append(arr)
    # print(len(arrs))
    #print(arrs[0].dtype)
    #print(arrs[0]['xl_hand'])
    name = f'a_{ID}_{class_name}_{score}'
    try:
        
        sk_viewer(arrs, i, name, class_name, 0, 1)
    except:
        print(name)
        pass
    #print(name)